<a href="https://colab.research.google.com/github/PranavM877/Cat-VS-Dog/blob/main/cat_vs_dog_cnn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install opendatasets

In [5]:
import opendatasets as od

url = "https://www.kaggle.com/datasets/anthonytherrien/dog-vs-cat"
od.download(url, data_dir='.')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: pranavmehta877
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/anthonytherrien/dog-vs-cat


100%|██████████| 360M/360M [00:04<00:00, 91.6MB/s]


In [6]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

# PARAMETERS
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 10
DATA_DIR = '/content/dog-vs-cat/animals' # Correct path to your sorted images

# Step 1: Set up data generators with automatic splitting
print("Setting up data generators...")

# Use one generator for both training and validation, and let it split the data
# Apply data augmentation only to the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # Create an 80% train, 20% validation split
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# The validation generator should not have augmentations, only rescaling
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


# Create the training generator from the 'training' subset
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'  # Specify this is the training set
)

# Create the validation generator from the 'validation' subset
validation_generator = validation_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'  # Specify this is the validation set
)

# Step 2: Create and compile the model
print("\nCreating model...")
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=IMG_SIZE + (3,)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

# Step 3: Train the model
print("\nTraining model...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    verbose=1
)

# Step 4: Save the trained model and preprocessing configuration
print("\nSaving model and preprocessing pipeline...")
model.save('dog_cat_model.h5')

preprocess_config = {
    'IMG_SIZE': IMG_SIZE,
    'class_indices': train_generator.class_indices
}
with open('preprocess_pipeline.pkl', 'wb') as f:
    pickle.dump(preprocess_config, f)

print("\n--- Training completed successfully! ---")

Setting up data generators...
Found 800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.

Creating model...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,939,329 (49.36 MB)

...

 Non-trainable params: 0 (0.00 B)


Training model...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.5074 - loss: 0.9263 - val_accuracy: 0.5550 - val_loss: 0.6636
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.5971 - loss: 0.6668 - val_accuracy: 0.7000 - val_loss: 0.6467
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.6600 - loss: 0.6505 - val_accuracy: 0.6850 - val_loss: 0.5403
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.7187 - loss: 0.5535 - val_accuracy: 0.7300 - val_loss: 0.4985
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.7370 - loss: 0.5286 - val_accuracy: 0.8400 - val_loss: 0.3451
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.8105 - loss: 0.4117 - val_accuracy: 0.9550 - val_loss: 0.2690
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.8345 - loss: 0.3790 - val_accuracy: 0.9600 - val_loss: 0.1720
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 44s 2s/step - accuracy: 0.8689 - loss: 0.2946 - val_accuracy: 0.9700 - val_loss:


Saving model and preprocessing pipeline...

--- Training completed successfully! ---


In [7]:
from google.colab import files
files.download('preprocess_pipeline.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
from google.colab import files
files.download('dog_cat_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>